In [ ]:
application "tropical";

In [ ]:
sub harnack_curve_tmp {
  my $n = shift;
  my @monoms;
  my @weights;
  my @signs;
  foreach my $i (0 .. $n) {
    foreach my $j (0 .. $n-$i) {
      push @weights, ($i*$j+$i*$i+$j*$j);
      push @monoms, [$n-$i-$j, $i, $j];
      push @signs, ($i*$j+$i+$j)%2;
    }
  }

  my $m = new Matrix<Int>(\@monoms);
  my $c = new Vector<TropicalNumber<Min>>(\@weights);
  my $s = new Array<Bool>(\@signs);
  my $h = new Hypersurface<Min>(MONOMIALS=>$m, COEFFICIENTS=>$c);
  $h->PATCHWORK(SIGNS=>$s);
  return $h;
}

In [ ]:
$h = harnack_curve_tmp(5);

In [ ]:
$h->properties;

In [ ]:
$comp = $h->COMPACTIFICATION;

In [ ]:
$rf = $h->PATCHWORK->REAL_FACETS;

In [ ]:
print $rf->[0];

In [ ]:
print $h->PATCHWORK->SIGNS;

In [ ]:
print rows_labeled($h->MAXIMAL_POLYTOPES);

In [ ]:
$decor = $comp->DECORATION;
$facets = $h->MAXIMAL_POLYTOPES;
$real_facets = $h->PATCHWORK->REAL_FACETS;
$nm = new NodeMap<Directed, Set<Int>>($comp->ADJACENCY);
foreach my $node (@{$comp->nodes_of_rank(2)}){
    my $real = $decor->[$node]->realisation;
    # Only works since the thing is pure
    my $facetno = -1;
    for(my $i=0; $i<$facets->rows; $i++){
        if($facets->[$i] == $real){
            $facetno = $i;
            last;
        }
    }
    print $node," ",$real," ",$facetno,": ";

    for(my $i = 0; $i<$real_facets->rows; $i++){
        if($real_facets->[$i]->contains($facetno)){
            print $i,",";
            $nm->[$node] += $i;
        }
    }
    print "\n";
    
}

In [ ]:
foreach my $node (@{$comp->nodes_of_rank(1)}){
    # print $comp->ADJACENCY->out_adjacent_nodes($node),"\n";
    foreach my $parent (@{$comp->ADJACENCY->out_adjacent_nodes($node)}){
        $nm->[$node] += $nm->[$parent];
    }
}
foreach my $node (@{$comp->nodes_of_rank(0)}){
    # print $comp->ADJACENCY->out_adjacent_nodes($node),"\n";
    foreach my $parent (@{$comp->ADJACENCY->out_adjacent_nodes($node)}){
        $nm->[$node] += $nm->[$parent];
    }
}

In [ ]:
$em = new EdgeMap<Directed, Matrix<GF2>>($comp->ADJACENCY);

In [ ]:
sub make_matrix {
    my($source, $target) = @_;
    # print "S: ",$source,"T: ",$target,"\n";
    my $result = unit_matrix<GF2>($source->size());
    my $remaps = new Set<Int>();
    my $remapt = new Set<Int>();
    my $i = 0;
    foreach my $s (@$source){
        $remaps += $i;
        if($target->contains($s)){
            $remapt += $i;
        }
        $i++;
    }
    return $result->minor($remapt, $remaps);
}

In [ ]:
for (my $e=entire(edges($comp->ADJACENCY)); $e; ++$e) {
    $em->[$$e] = make_matrix($nm->[$e->from_node], $nm->[$e->to_node]);
}

In [ ]:
print $comp->TOP_NODE;

In [ ]:
$top = $comp->TOP_NODE;
foreach my $node (@{$comp->ADJACENCY->in_adjacent_nodes($top)}){
    $em->edge($node, $top) = zero_matrix<GF2>($nm->[$node]->size(),0);
}

In [ ]:
$Polymake::User::Verbose::cpp = 1;
$chain = fan::build_full_chain($comp, $comp->ORIENTATIONS, $em, true);

In [ ]:
print $comp->ORIENTATIONS;

In [ ]:
print $chain;

In [ ]:
print $chain->type->full_name;

In [ ]:
print topaz::betti_numbers<GF2>($chain);

In [ ]:
$hr = $h->PATCHWORK->realize("uniform");

In [ ]:
print $hr->HOMOLOGY;

# A larger example
The previous examples had a relatively small dimension, so we could not study the effects of having different non-trivial sedentarities.

In [ ]:
$trop = new Hypersurface<Max>(POLYNOMIAL=>toTropicalPolynomial("max(a,b,c,d,e)"));

In [ ]:
print $trop->VERTICES;

In [ ]:
print rows_labeled($trop->COMPACTIFICATION->DECORATION);

In [ ]:
$p = $trop->PATCHWORK(SIGNS=>[1,1,1,1,1]);

In [ ]:
print transpose($p->REAL_FACETS);

In [ ]:
$v = new Matrix<Int>($trop->VERTICES);
$v = new Matrix<GF2>($v);
print $v;

The rays of the tropical hypersurface should be the rays of the normal fan of the simplex. However we do not know where and in which order they appear. The following method will label the rays accordingly. It is just an auxiliary method for building the right projection maps to the boundary.

In [ ]:
sub relabel {
    my($vert) = @_;
    my $d = $vert->cols();
    my $result = new Array<Int>($vert->rows());
    for(my $i=0; $i<$vert->rows(); $i++){
        if($vert->row($i)->[0] == 0){
            my $v = $vert->row($i)->slice(sequence(2,$d-2));
            for(my $j=0; $j<$d-2; $j++){
                if($v == -unit_vector($d-2,$j)){
                    # print "Found: $i $j\n";
                    $result->[$i] = $j;
                    last;
                }
            }
            if($v == ones_vector($d-2)){
                $result->[$i] = -1;
            }
        }
    }
    return $result;
}

In [ ]:
print relabel($trop->VERTICES);

There are two main cases for the sedentarity of a face: It can contain the all ones vector or not. For both cases we assemble projection and lifting matrices. The lifting is needed as we can build projections between faces with non-trivial sedentarity by going via the lifting to the parent face.

In [ ]:
sub get_gf2_projections {
    my($h) = @_;
    my $relabel = relabel($h->VERTICES);
    my $d = $h->VERTICES->cols() - 2;
    my $result = new Map<Set<Int>, Pair<Matrix<GF2>, Matrix<GF2>>>();
    my $comp = $h->COMPACTIFICATION;
    my $decor = $comp->DECORATION;
    for(my $node = entire(nodes($comp->ADJACENCY)); $node; ++$node){
        my $psed = $decor->[$$node]->sedentarity;
        print $$node,": $psed ";
        my $sed = new Set<Int>();
        foreach my $entry (@$psed){
            $sed += $relabel->[$entry];
        }
        my ($projection, $lifting);
        if($sed->contains(-1)){
            my $first = 0;
            while($sed->contains($first)){
                $first++;
            }
            $first < $rays->rows() or die "Could not find non-contained element";
            $projection = new Matrix<GF2>(unit_matrix<GF2>($d));
            $projection->col($first) = ones_vector<GF2>($d);
            $lifting = inv($projection);
            $sed -= -1;
            $sed += $first;
            $projection = $projection->minor(~$sed, All);
            $lifting = $lifting->minor(All, ~$sed);
        } else {
            $projection = unit_matrix<GF2>($d);
            $lifting = unit_matrix<GF2>($d);
            $projection = $projection->minor(~$sed, All);
            $lifting = $lifting->minor(All, ~$sed);
        }
        my $prod = $projection * $lifting;
        my $check = $prod == unit_matrix<GF2>($projection->rows);
        my $vert = $h->VERTICES->minor($psed, sequence(2,$d));
        $prod = $projection * transpose(new Matrix<GF2>(convert_to<Integer>($vert)));
        $check &= $prod == zero_matrix<GF2>($prod->rows, $prod->cols);
        print "Check: $check\n";
        if($check == 0){
            print "Proj:\n",$projection,"\n";
            print "Vert:\n",$vert,"\n";
            print "Prod:\n",$prod,"\n";
        }
        $result->{$psed} = new Pair<Matrix<GF2>, Matrix<GF2>>($projection, $lifting);
    }
    return $result;
}

In [ ]:
print get_gf2_projections($trop);

In [ ]:
print $trop->PATCHWORK->REAL_FACETS->rows;

In [ ]:
print $trop->PATCHWORK->REAL_FACETS;